In [1]:
# !pip install predictionguard
# !pip install langdetect
# !pip install langchain

In [2]:
import os
import random

from predictionguard import PredictionGuard
from getpass import getpass
import pandas as pd
from sklearn.model_selection import train_test_split
from langdetect import detect
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
import numpy as np
import time
from datetime import datetime


In [3]:
data = pd.read_csv('examples.csv')

In [4]:
data.head()

,Id,q1,q2,q3,q4,q5,q6,q7,q8,q9,...,q14,q15,q16,q17,q18,q19,q20,q21,q22,label
0,bfe3aa97-eb08-43d9-88a5-34ce93509321,6,6.0,1,2,2,2,2,2,1,...,2,2,1,2,1,2,1,"As a surviving child, the weekend seminar with...","""As a surviver chil going to the TAPS Seminar,...",3
1,3b47ee28-e621-4497-949b-0fa0a59044f7,0,4.0,1,2,2,2,0,1,1,...,2,2,0,1,1,2,2,NaN,Como hijo sobreviviente que asistió al Seminar...,3
2,e21808f4-2c7f-4cdf-90f5-daa7ccccc843,0,5.0,2,2,2,2,0,4,2,...,1,2,2,2,2,2,2,NaN,"""The TAPS Seminar was incredibly valuable for ...",4
3,59385c6a-5a11-4348-b0d1-c1f87b56cbd3,5,NaN,0,1,1,1,4,2,1,...,1,3,2,3,3,3,1,NaN,"""I attended the TAPS Seminar recently, and it ...",2
4,6a769ce6-900b-4490-9d67-6a9e8190b78c,0,0.0,1,2,2,2,2,2,1,...,1,2,2,1,1,2,1,"During our unforgettable weekend, we cherishin...",NaN,3


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 24 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Id      50 non-null     object 
 1   q1      50 non-null     int64  
 2   q2      34 non-null     float64
 3   q3      50 non-null     int64  
 4   q4      50 non-null     int64  
 5   q5      50 non-null     int64  
 6   q6      50 non-null     int64  
 7   q7      50 non-null     int64  
 8   q8      50 non-null     int64  
 9   q9      50 non-null     int64  
 10  q10     50 non-null     int64  
 11  q11     50 non-null     int64  
 12  q12     50 non-null     int64  
 13  q13     50 non-null     int64  
 14  q14     50 non-null     int64  
 15  q15     50 non-null     int64  
 16  q16     50 non-null     int64  
 17  q17     50 non-null     int64  
 18  q18     50 non-null     int64  
 19  q19     50 non-null     int64  
 20  q20     50 non-null     int64  
 21  q21     29 non-null     object 
 22  q22 

In [6]:
data.isnull().sum()

Id        0
q1        0
q2       16
q3        0
q4        0
q5        0
q6        0
q7        0
q8        0
q9        0
q10       0
q11       0
q12       0
q13       0
q14       0
q15       0
q16       0
q17       0
q18       0
q19       0
q20       0
q21      21
q22      28
label     0
dtype: int64

In [7]:
pg_access_token = getpass('Enter your Prediction Guard access api key: ')
os.environ['PREDICTIONGUARD_API_KEY'] = pg_access_token

Enter your Prediction Guard access api key: ········


In [8]:
client = PredictionGuard()

In [9]:
data.label.value_counts()

label
2    11
3    10
4     9
6     7
1     7
5     6
Name: count, dtype: int64

In [10]:
# # Number of samples needed per class for training
# samples_per_class = 4  # Adjust as needed for balanced sampling

# # Create a balanced training set with classes 1 to 6
# train_data = data.groupby('label').sample(n=samples_per_class, random_state=42)

# # Get the remaining observations as the test set
# test_data = data.drop(train_data.index)

# # Display the class distribution for verification
# print("Training set class distribution:")
# print(train_data['label'].value_counts().sort_index())

# print("\nTest set class distribution:")
# print(test_data['label'].value_counts().sort_index())


In [11]:
train_data, test_data = train_test_split(data, test_size=0.6, random_state=42)

print("Train data shape:", train_data.shape)
print("Test data shape:", test_data.shape)

Train data shape: (20, 24)
Test data shape: (30, 24)


In [12]:
train_data.label.value_counts()

label
2    6
3    5
4    3
1    3
6    2
5    1
Name: count, dtype: int64

In [13]:
# Function to format a single row without including the Id in the response dictionary
def format_response(row):
    formatted_response = {}
    for question, response in row.items():
        if question != "Id":  # Exclude the Id field from the response
            formatted_response[question] = response
    return formatted_response

In [14]:
# def detect_language(text):
#     try:
#         return detect(text)
#     except:
#         return None  # in case detection fails

In [15]:
# q22_language = data['q22'].apply(detect_language)
# q_22_unique_languages = q22_language.unique()

# q21_language = data['q21'].apply(detect_language)
# q_21_unique_languages = q21_language.unique()

In [16]:
# Generate a list of formatted responses for each row, excluding Id
example_responses = [format_response(row) for _, row in train_data.iterrows()]

In [17]:
test_label = test_data['label']
test_responses = test_data.drop(columns=['label'])

In [18]:
formatter_template = """
Q1:    {q1}
Q2:    {q2}
Q3:    {q3}
Q4:    {q4}
Q5:    {q5}
Q6:    {q6}
Q7:    {q7}
Q8:    {q8}
Q9:    {q9}
Q10:   {q10}
Q11:   {q11}
Q12:   {q12}
Q13:   {q13}
Q14:   {q14}
Q15:   {q15}
Q16:   {q16}
Q17:   {q17}
Q18:   {q18}
Q19:   {q19}
Q20:   {q20}
Q21:   {q21}
Q22:   {q22}
Label: {label}
"""


prompt = PromptTemplate(
    input_variables=["q1", "q2", "q3", "q4", "q5", "q6", "q7", "q8", "q9", "q10",
    "q11", "q12", "q13", "q14", "q15", "q16", "q17", "q18", "q19",
    "q20", "q21", "q22", "label"],
    template=formatter_template,
)


In [19]:
few_examples = example_responses

In [20]:
examples = []
for ex in few_examples:
  example = {}
  for i in range(1,23):
    question_key = f"q{i}"
    example[question_key] = ex[question_key]

  example["label"] = ex["label"]
  examples.append(example)

In [21]:
few_shot_prompt = FewShotPromptTemplate(
    # This is the demonstration data we want to insert into the prompt.
    examples=examples,
    example_prompt=prompt,
    example_separator="",
    # This is the boilerplate portion of the prompt corresponding to
    # the prompt task instructions.
    prefix="""TAPS has developed an initial version of what they call a "Journey Map", which maps the major stages and tasks associated with a survivor's grief journey. TAPS evaluates a survivor's grief journey and categorizes them into one of the six stages based on a survey:

    1. Immediate Grief, Shock & Emotion (Label 1)
    Distinguishing Characteristics:
    High scores on feeling alone and scared about the future (q11, q14), reflecting initial shock.
    Low ratings on hope and positive outlook (q6, q9), with limited ability to cope.
    Responses may mention struggling to manage emotions or family responsibilities.
    Indicators show a sense of isolation and emotional overwhelm, with little connection to community or purpose.
    2. Navigating Family Relationships (Label 2)
    Distinguishing Characteristics:
    Mixed or neutral responses around social connection (q3) and support, showing varying family tensions.
    Struggle with understanding and empathy within family dynamics, as indicated by responses in family-related support areas (q4, q5).
    Responses highlight different grief phases among family members and feelings of tension.
    Indicators reflect internal family conflict and a need for guidance on navigating family relationships during grief.
    3. Learning to Process Grief (Label 3)
    Distinguishing Characteristics:
    Positive shift in responses around coping (q4) and understanding grief (q5), indicating engagement with grief processing.
    Moderate-to-positive feedback on social connection (q3), reflecting openness to peer support.
    Survivors begin to express interest in learning coping tools and exploring emotions.
    Indicators show a transitional stage, where individuals are beginning to process grief rather than being overwhelmed by it.
    4. Moments That Matter (Label 4)
    Distinguishing Characteristics:
    Heightened emotions around anniversaries, holidays, or special events, which may be mentioned in responses (q21, q22).
    Mixed feelings on social connection, with both positive and sensitive responses, highlighting the unique challenges of these times (q7).
    Indicators reflect a renewed experience of grief during specific dates, with support needed to navigate these emotional peaks.
    5. Feeling Immersed, Connected & Seen (Label 5)
    Distinguishing Characteristics:
    High ratings in community connection and comfort (q3, q6, q17), showing strong ties to support networks.
    Positive responses on outlook and purpose, with survivors feeling seen and less alone.
    Indicators reflect a sense of belonging and reduced isolation, distinguishing this as a stage of community immersion and emotional visibility.
    6. New Growth & Purpose (Label 6)
    Distinguishing Characteristics:
    High scores on goal setting, positive outlook, and strength (q9, q10, q16), indicating resilience.
    Responses may express readiness to mentor or volunteer, showing a shift toward giving back (q21, q22).
    Indicators reflect a healthy and stable point in the grief journey, marked by a desire to help others and contribute to the community.

    Here are the relevant survey quesitons and options for journey stage labeling:

    3. This seminar helped me to feel socially connected
    Disagree
    Neither agree nor disagree
    Agree

    4. This seminar taught me new ways to cope with my grief
    Disagree
    Neither agree nor disagree
    Agree

    5. This seminar helped me to better understand my grief
    Disagree
    Neither agree nor disagree
    Agree

    6. This seminar has given me hope for the future
    Disagree
    Neither agree nor disagree
    Agree

    7. Based on your experience at the TAPS [SEMINAR NAME], how likely are you to continue to connect with TAPS virtually / at in-person programs?
    Extremely likely / Very likely
    Somewhat likely
    Neither likely nor unlikely / Neutral
    Somewhat unlikely
    Extremely unlikely / Not at all likely

    9. I have a positive outlook toward life
    Strongly Disagree
    Disagree
    Agree
    Strongly Agree

    10. I have short and/or long range goals
    Strongly Disagree
    Disagree
    Agree
    Strongly Agree

    11. I feel all alone
    Strongly Disagree
    Disagree
    Agree
    Strongly Agree

    12. I can see possibilities in the midst of difficulties
    Strongly Disagree
    Disagree
    Agree
    Strongly Agree

    13. I have faith that gives me comfort
    Strongly Disagree
    Disagree
    Agree
    Strongly Agree

    14. I feel scared about my future
    Strongly Disagree
    Disagree
    Agree
    Strongly Agree

    15. I can recall happy/joyful times
    Strongly Disagree
    Disagree
    Agree
    Strongly Agree

    16. I have deep inner strength
    Strongly Disagree
    Disagree
    Agree
    Strongly Agree

    17. I am able to give and receive care/love
    Strongly Disagree
    Disagree
    Agree
    Strongly Agree

    18. I have a sense of direction
    Strongly Disagree
    Disagree
    Agree
    Strongly Agree

    19. I believe that each day has potential
    Strongly Disagree
    Disagree
    Agree
    Strongly Agree

    20. I feel my life has value and worth
    Strongly Disagree
    Disagree
    Agree
    Strongly Agree

    21. Please share with TAPS your favorite moment of the weekend? Did you have a breakthrough moment this weekend you would like to share?

    22. Please share any additional feedback you have regarding your TAPS Seminar experience.

    Classify the responses to a label corrosponding to the above 6 phases of the TAPS journey map.

    Below are some classified examples:
    """,
    # The suffix of the prompt is where we will put the output indicator
    # and define where the "on-the-fly" user input would go.
    suffix="\nResponses: {input}\nWhat's the label for this response?",
    input_variables=["input"],
)

In [22]:
myprompt = few_shot_prompt.format(input=format_response(test_responses.drop(columns = 'Id').iloc[0]))
print(myprompt)

TAPS has developed an initial version of what they call a "Journey Map", which maps the major stages and tasks associated with a survivor's grief journey. TAPS evaluates a survivor's grief journey and categorizes them into one of the six stages based on a survey:

    1. Immediate Grief, Shock & Emotion (Label 1)
    Distinguishing Characteristics:
    High scores on feeling alone and scared about the future (q11, q14), reflecting initial shock.
    Low ratings on hope and positive outlook (q6, q9), with limited ability to cope.
    Responses may mention struggling to manage emotions or family responsibilities.
    Indicators show a sense of isolation and emotional overwhelm, with little connection to community or purpose.
    2. Navigating Family Relationships (Label 2)
    Distinguishing Characteristics:
    Mixed or neutral responses around social connection (q3) and support, showing varying family tensions.
    Struggle with understanding and empathy within family dynamics, as indi

In [23]:
# result = client.completions.create(
#     model="Hermes-3-Llama-3.1-70B",
#     prompt=myprompt,
#     max_tokens=2
# )

In [24]:
# result

In [25]:
test_data['formatted_response'] = test_data.drop(columns = 'label').apply(format_response, axis=1)

In [26]:
test_data.formatted_response.iloc[1]

{'q1': 1,
 'q2': 2.0,
 'q3': 2,
 'q4': 2,
 'q5': 2,
 'q6': 2,
 'q7': 0,
 'q8': 1,
 'q9': 2,
 'q10': 2,
 'q11': 1,
 'q12': 2,
 'q13': 2,
 'q14': 1,
 'q15': 2,
 'q16': 2,
 'q17': 2,
 'q18': 2,
 'q19': 2,
 'q20': 2,
 'q21': nan,
 'q22': '"Aprecié la capacidad del Seminario TAPS de abordar las experiencias de duelo renovado que enfrento cada año durante momentos significativos y aniversarios del fallecimiento de mi ser querido. Las conexiones compartidas y el apoyo de las trabajados de la Unidad Familiar Sobreviviente proporcionaron una orientación invaluable sobre cómo navegar en los momentos especiales en la vida de nuestros hijos. Aunque estos momentos pueden ser amargos y dulces, las herramientas y recursos que obtuvimos del seminario han fortalecido nuestra familia para poder honrar la memoria de nuestro ser querido juntos. ¡Muy altamente recomendado el programa TAPS para cualquier familia que atravesa desafíos similares! ¡Siguen haciendo un trabajo excepcional!"'}

In [27]:
def classify_response(response):
    try:
        myprompt = few_shot_prompt.format(input=response)
        result = client.completions.create(
            model="Hermes-3-Llama-3.1-70B",
            prompt=myprompt,
            temperature=0.1
        )

        for char in result['choices'][0]['text'].strip():
            if char.isdigit():
                first_digit = char
                break

        time.sleep(10)
        return int(first_digit)
    except Exception as e:
        print(f"Error: {e}. Skipping this response.")
        return None  # Return None or a specific default value for failed cases

In [28]:
# test_data['predicted_label'] = test_data.formatted_response.apply(classify_response)

In [29]:
def ensemble_classify(df, n_run = 5):
    for i in range(n_run):
        df['predicted_label'+f'_{i}'] = df.formatted_response.apply(classify_response)
    df['majority_label'] = df[['predicted_label'+f'_{i}' for i in range(n_run)]].mode(axis=1)[0]

In [30]:
# for i in range(5):
#   test_data['predicted_label'+f'_{i}'] = test_data.formatted_response.apply(classify_response)

In [31]:
# test_data['majority_label'] = test_data[['predicted_label'+f'_{i}' for i in range(5)]].mode(axis=1)[0]

In [32]:
# # prompt: compute accuracy using label and predicted label in test_data

# from sklearn.metrics import accuracy_score, f1_score

# # Assuming test_data has 'label' and 'predicted_label' columns
# y_true = test_data['label']
# # y_pred = test_data['majority_label']
# y_pred = test_data['predicted_label']

# accuracy = accuracy_score(y_true, y_pred)
# print(f"Accuracy: {accuracy}")

In [33]:
# f1_score(y_true, y_pred, average='weighted')

In [34]:
# test_data

In [35]:
# test_data[['label', 'majority_label']]

In [36]:
test = pd.read_csv('test.csv')

In [37]:
test.shape

(2950, 23)

In [38]:
test['formatted_response'] = test.apply(format_response, axis=1)

In [39]:
save_path = 'test_results.csv'

In [40]:
if os.path.exists(save_path):
    test = pd.read_csv(save_path)
else:
    test['predicted_label'] = None  # Initialize column if not already in the CSV

while test['predicted_label'].isna().any():
    for idx, row in test.iterrows():
        if pd.isna(row['predicted_label']):  # Skip if label already exists
            label = classify_response(row['formatted_response'])
            test.at[idx, 'predicted_label'] = label

        # Save progress every 10 rows
        if idx % 10 == 0:
            test.to_csv(save_path, index=False)
            print(f"Progress saved at row {idx}")
    test.to_csv(save_path, index=False)
    print("Round completed. Checking for unclassified rows...")

# Final save after loop completion
test.to_csv(save_path, index=False)
print("Classification completed and progress saved.")

Classification completed and progress saved.


In [41]:
test

,Id,q1,q2,q3,q4,q5,q6,q7,q8,q9,...,q15,q16,q17,q18,q19,q20,q21,q22,formatted_response,predicted_label
0,a2c83973-8e81-45c9-bb84-37fa8a8b637c,3,NaN,0,1,1,1,4,4,1,...,1,1,0,0,0,0,NaN,NaN,"{'q1': 3, 'q2': nan, 'q3': 0, 'q4': 1, 'q5': 1...",1.0
1,78bca2d4-8824-45ed-80c9-72ef0e4389c6,1,6.0,2,2,2,1,0,0,2,...,2,2,2,2,2,2,NaN,"""As someone navigating the renewed experience ...","{'q1': 1, 'q2': 6.0, 'q3': 2, 'q4': 2, 'q5': 2...",4.0
2,69fc99e5-a555-4b3e-a2e2-6c2a4ce45f36,3,2.0,1,2,2,2,2,2,2,...,2,2,1,1,2,2,NaN,Attendig the TAPS seminar as a survivin childe...,"{'q1': 3, 'q2': 2.0, 'q3': 1, 'q4': 2, 'q5': 2...",3.0
3,d5d506d9-29bb-404d-9061-3db9ae92d711,4,NaN,0,2,1,2,2,0,0,...,0,0,2,3,0,2,"""I had a breakthrough at the weekend seminar d...",The TAPS Seminiar give me a unforgittable chan...,"{'q1': 4, 'q2': nan, 'q3': 0, 'q4': 2, 'q5': 1...",2.0
4,d0796afc-055d-4ea8-9552-336ad0442de5,1,NaN,0,1,1,0,1,4,1,...,0,1,1,1,0,0,"""During the weekend seminar with TAPS, my favo...","The TAPS Seminar ws ee-owening, challinging, a...","{'q1': 1, 'q2': nan, 'q3': 0, 'q4': 1, 'q5': 1...",1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2945,1885d9ad-c858-4cc9-877b-491e912386a5,7,7.0,2,2,2,2,0,0,2,...,2,2,2,2,2,2,NaN,"""Attending the TAPS Seminar has been a profoun...","{'q1': 7, 'q2': 7.0, 'q3': 2, 'q4': 2, 'q5': 2...",4.0
2946,396a3f34-6f62-4785-9707-15f22e1f23b9,4,NaN,1,2,0,2,4,2,2,...,2,0,0,1,2,3,"""Tuve un momento de revelación durante la sesi...","Durante el Seminario TAPS, enfrenté una dificu...","{'q1': 4, 'q2': nan, 'q3': 1, 'q4': 2, 'q5': 0...",2.0
2947,5ad78da8-4a84-4685-b2b3-7e49590915f7,0,NaN,1,2,0,0,1,2,3,...,3,2,0,0,2,3,NaN,NaN,"{'q1': 0, 'q2': nan, 'q3': 1, 'q4': 2, 'q5': 0...",1.0
2948,55188d8a-8895-4289-acf5-9606972dca8e,0,NaN,2,0,1,2,1,0,3,...,3,3,3,3,3,2,NaN,"The TAPS Seminar transformed me, empowered me ...","{'q1': 0, 'q2': nan, 'q3': 2, 'q4': 0, 'q5': 1...",6.0


In [42]:
results = test[['Id', 'predicted_label']]

results.columns = ['Id', 'Label']

results.to_csv('submission.csv', index = False)